## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [73]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

Version branch cambios 18 enero

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * `estado_vacunacion_.csv` , última versión del mismo con el acumulado de todos los informes publicados
  * `estado_vacunacion_yearmonthday.csv`, con la versión del día en cuestión, solo con la información de ese día

In [74]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=2,day=19)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en red en formato ods

In [75]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [76]:
try:
    datos_row = pd.read_excel(enlace)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210311.ods


Le echamos un vistazo para asegurar que ha ido todo bien

In [77]:
datos_row

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Dosis entregadas AstraZeneca (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas vacunadas(pauta completada),Fecha de la última vacuna registrada (2)
0,Andalucía,820260,73300,303300,1196860,1000998,0.836353,292741,2021-03-10
1,Aragón,147735,12600,47600,207935,166351,0.800014,53228,2021-03-10
2,Asturias,144815,12700,36600,194115,145833,0.751271,48210,2021-03-10
3,Baleares,76410,6500,41900,124810,95435,0.764642,26495,2021-03-10
4,Canarias,167280,14300,78000,259580,194364,0.748763,58482,2021-03-10
5,Cantabria,70935,5900,20800,97635,72537,0.742941,22624,2021-03-10
6,Castilla y Leon,324975,27300,85800,438075,339992,0.776105,110038,2021-03-10
7,Castilla La Mancha,215145,18600,73300,307045,223931,0.729310,76016,2021-03-10
8,Cataluña,735380,63400,278900,1077680,835931,0.775676,218613,2021-03-10
9,C. Valenciana,395650,31700,181300,608650,448396,0.736706,130387,2021-03-10


..hacemos una copia para hacerle las transformaciones necesarias:

In [78]:
datos_hoy = datos_row.copy()

Primero renombramos columnas, añadimos luego una columna con la fecha del informe `date_pub`, y finalmente añadimos fuente y origen de datos para asegurar trazabilidad de los datos:

In [79]:
datos_hoy.columns=['ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas','Dosis administradas','% sobre entregadas','Total pauta completada','Fecha de la ultima vacuna registrada']

In [80]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace


vamos a convertir el formato de `Fecha de la ultima vacuna registrada` para que aparezca como día/mes/año sin añadir ceros de pad..de ahi el signo menos en el caso de dia y mes (%-d/%-m). La excepción se añade para la línea de Totales, que no lleva fecha asociada:

In [81]:
def convierte(x):
    campo =  x['Fecha de la ultima vacuna registrada']
    try:
        fecha = campo.strftime('%-d/%-m/%y')
        return fecha
    except:
        return campo

In [82]:
datos_hoy['Fecha de la ultima vacuna registrada'] = datos_hoy.apply(convierte,axis=1)

..y ordenamos..

In [83]:
datos_hoy

,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,date_pub,source_name,source
0,Andalucía,820260,73300,303300,1196860,1000998,0.836353,292741,10/3/21,11/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,Aragón,147735,12600,47600,207935,166351,0.800014,53228,10/3/21,11/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,Asturias,144815,12700,36600,194115,145833,0.751271,48210,10/3/21,11/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,Baleares,76410,6500,41900,124810,95435,0.764642,26495,10/3/21,11/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,Canarias,167280,14300,78000,259580,194364,0.748763,58482,10/3/21,11/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,Cantabria,70935,5900,20800,97635,72537,0.742941,22624,10/3/21,11/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,Castilla y Leon,324975,27300,85800,438075,339992,0.776105,110038,10/3/21,11/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,Castilla La Mancha,215145,18600,73300,307045,223931,0.729310,76016,10/3/21,11/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,Cataluña,735380,63400,278900,1077680,835931,0.775676,218613,10/3/21,11/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,C. Valenciana,395650,31700,181300,608650,448396,0.736706,130387,10/3/21,11/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [84]:
datos_hoy=datos_hoy[['date_pub','ccaa','Dosis entregadas Pfizer',
                    'Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas',
                    'Dosis administradas','% sobre entregadas',
                    'Total pauta completada',
                    'Fecha de la ultima vacuna registrada','source_name','source']]

Hemos visto algún informe en el que se añaden espacios al nombre de Comunidad, nos aseguramos aquí de eliminarlos en caso de que vuelvan a aparecer:

In [85]:
datos_hoy['ccaa'] = datos_hoy['ccaa'].apply(lambda x : x.rstrip().lstrip())

In [86]:
datos_hoy['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

In [87]:
datos_hoy

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,source_name,source
0,11/3/21,Andalucía,820260,73300,303300,1196860,1000998,0.836353,292741,10/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,11/3/21,Aragón,147735,12600,47600,207935,166351,0.800014,53228,10/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,11/3/21,Asturias,144815,12700,36600,194115,145833,0.751271,48210,10/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,11/3/21,Baleares,76410,6500,41900,124810,95435,0.764642,26495,10/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,11/3/21,Canarias,167280,14300,78000,259580,194364,0.748763,58482,10/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,11/3/21,Cantabria,70935,5900,20800,97635,72537,0.742941,22624,10/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,11/3/21,Castilla y Leon,324975,27300,85800,438075,339992,0.776105,110038,10/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,11/3/21,Castilla La Mancha,215145,18600,73300,307045,223931,0.729310,76016,10/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,11/3/21,Cataluña,735380,63400,278900,1077680,835931,0.775676,218613,10/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,11/3/21,C. Valenciana,395650,31700,181300,608650,448396,0.736706,130387,10/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [88]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [89]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [90]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
935,10/3/21,Navarra,69765.0,5900.0,23600.0,99265,74743,0.752964,24805.0,NaN,9/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
936,10/3/21,País Vasco,208080.0,16000.0,79600.0,303680,189949,0.625491,55272.0,NaN,9/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
937,10/3/21,Ceuta,5490.0,600.0,3100.0,9190,8262,0.899021,2443.0,NaN,9/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
938,10/3/21,Melilla,4320.0,600.0,3100.0,8020,6558,0.817706,1953.0,NaN,9/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


concatenamos...

In [91]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [92]:
datos_acumulados.columns

Index(['date_pub', 'ccaa', 'Dosis entregadas Pfizer',
       'Dosis entregadas Moderna', 'Dosis entregadas AstraZeneca',
       'Dosis entregadas', 'Dosis administradas', '% sobre entregadas',
       'Total pauta completada', 'Última fecha de actualización de datos',
       'Fecha de la ultima vacuna registrada', 'source_name', 'source'],
      dtype='object')

In [93]:
datos_acumulados = datos_acumulados[['date_pub', 'ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna',
                                     'Dosis entregadas AstraZeneca',
                                     'Dosis entregadas', 'Dosis administradas','% sobre entregadas',
                                     'Total pauta completada','Última fecha de actualización de datos',
                                     'Fecha de la ultima vacuna registrada','source_name', 'source']]

In [94]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,11/3/21,Navarra,69765.0,5900.0,23600.0,99265,75059,0.756148,24831.0,NaN,10/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
956,11/3/21,País Vasco,208080.0,16000.0,79600.0,303680,200677,0.660817,57875.0,NaN,10/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
957,11/3/21,Ceuta,5490.0,600.0,3100.0,9190,8571,0.932644,2533.0,NaN,10/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
958,11/3/21,Melilla,4320.0,600.0,3100.0,8020,6851,0.854239,2022.0,NaN,10/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [95]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

y volvemos a subirlo..

In [96]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)